### Primero importamos todas las librerias que vamos a necesitar

In [1]:
#!pip install redshift_connector

In [2]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import os
from dotenv import load_dotenv
import redshift_connector

### A través de load_dotenv, ocultamos los datos senciles mediantes variables entorno

In [3]:
load_dotenv()
host = os.getenv('REDSFHIT_HOST')
port = os.getenv('REDSHIFT_PORT')
database = os.getenv('REDSHIFT_DATABASE')
user = os.getenv('REDSHIFT_USER')
password = os.getenv('REDSHIFT_PASSWORD')
schema = os.getenv('REDSHIFT_SCHEMA')

#engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

### creamos la función redshift_connection para utilizarlo en otra instancia del proyecto

In [6]:
# def redshift_connection(host, port, database, user, password):
#     print('Connecting to Redshfift Cluster.....')
#     conn = redshift_connector.connect(
#         host = host,
#         port = port,
#         database = database,
#         user = user,
#         password = password 
#     )
    
#     engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}//{database}')

#     print(f'Succesfully connected to Redshift cluster: {host}')
#     return conn, engine

In [7]:
# conn = redshift_connection(host, port, database, user, password)

Connecting to Redshfift Cluster.....
Succesfully connected to Redshift cluster: data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com


In [ ]:
# def execute_sql(conn, query, method):
#     try:
#         cursor = conn.cursor()
#         conn.autocommit = True
#         cursor.execute(query)

#         if method == 'classic':
#             res = cursor.fetchall()
#             return res
#         elif method == 'df':
#             df = pd.DataFrame(cursor.fetchall())
#             return df
#     except Exception as e:
#         print(f'Failed with error messaga: {e}')
#         raise e
#     finally:
#         cursor.close()
#         conn.close()

### Hacemos nuestra conexión con nuestra API y mostramos los resultados

In [8]:
link = "https://api.themoviedb.org/3/movie/top_rated?api_key=59a68b6170002814047c923975011cff"
response = requests.get(link)

print(response.text)

{"page":1,"results":[{"adult":false,"backdrop_path":"/tmU7GeKVybMWFButWEGl2M4GeiP.jpg","genre_ids":[18,80],"id":238,"original_language":"en","original_title":"The Godfather","overview":"Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.","popularity":110.79,"poster_path":"/3bhkrj58Vtu7enYsRolD1fZdja1.jpg","release_date":"1972-03-14","title":"The Godfather","video":false,"vote_average":8.7,"vote_count":18684},{"adult":false,"backdrop_path":"/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg","genre_ids":[18,80],"id":278,"original_language":"en","original_title":"The Shawshank Redemption","overview":"Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he pu

In [9]:
response.json()["results"]

[{'adult': False,
  'backdrop_path': '/tmU7GeKVybMWFButWEGl2M4GeiP.jpg',
  'genre_ids': [18, 80],
  'id': 238,
  'original_language': 'en',
  'original_title': 'The Godfather',
  'overview': 'Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.',
  'popularity': 110.79,
  'poster_path': '/3bhkrj58Vtu7enYsRolD1fZdja1.jpg',
  'release_date': '1972-03-14',
  'title': 'The Godfather',
  'video': False,
  'vote_average': 8.7,
  'vote_count': 18684},
 {'adult': False,
  'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg',
  'genre_ids': [18, 80],
  'id': 278,
  'original_language': 'en',
  'original_title': 'The Shawshank Redemption',
  'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy D

### Armamos un dataframe con la información traida de nuestra API

In [10]:
data = pd.DataFrame(response.json()["results"])[['id','title','popularity','release_date','vote_average','vote_count']]
data.head()

,id,title,popularity,release_date,vote_average,vote_count
0,238,The Godfather,110.790,1972-03-14,8.7,18684
1,278,The Shawshank Redemption,115.364,1994-09-23,8.7,24660
2,240,The Godfather Part II,84.578,1974-12-20,8.6,11298
3,424,Schindler's List,70.449,1993-12-15,8.6,14604
4,19404,Dilwale Dulhania Le Jayenge,26.473,1995-10-20,8.6,4256


### Verificamos si tenemos o no duplicados

In [11]:
data.duplicated().sum()

0

### Finalmente cargamos los datos de nuestro DataFrame a Amazon Redshift

In [13]:
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

data = data.reset_index(drop=True)
df = data.to_sql('peliculas', engine, if_exists="replace", index=False)